In [1]:
from bs4 import BeautifulSoup
import urllib.request
import requests
from pprint import pprint
import pandas as pd
import json
import copy
from datetime import datetime, timezone
from google.cloud import storage
import gcsfs
import re

In [2]:
storage_client = storage.Client(project="ytone-430507")
today = datetime.now()
table_name = "douyin_price"
api_name = "handler_post"

In [3]:
processing_blobs = [
{
    "blob": blob,
    "date": datetime.strptime(blob.name.split('/')[1], "%Y-%m-%d"),
    "batch_number": int(blob.name.split('/')[-1].replace(".json", "").split("_")[-1]),
} for blob in storage_client.list_blobs("0_raw_data",prefix="2_xingtu/") if api_name in blob.name]
processing_blobs

[{'blob': <Blob: 0_raw_data, 2_xingtu/2024-09-11/handler_post_0911_0.json, 1726037777115120>,
  'date': datetime.datetime(2024, 9, 11, 0, 0),
  'batch_number': 0},
 {'blob': <Blob: 0_raw_data, 2_xingtu/2024-09-12/handler_post_0912_0.json, 1726117857045627>,
  'date': datetime.datetime(2024, 9, 12, 0, 0),
  'batch_number': 0},
 {'blob': <Blob: 0_raw_data, 2_xingtu/2024-09-12/handler_post_0912_1.json, 1726117860404297>,
  'date': datetime.datetime(2024, 9, 12, 0, 0),
  'batch_number': 1},
 {'blob': <Blob: 0_raw_data, 2_xingtu/2024-09-12/handler_post_0912_10.json, 1726124693780255>,
  'date': datetime.datetime(2024, 9, 12, 0, 0),
  'batch_number': 10},
 {'blob': <Blob: 0_raw_data, 2_xingtu/2024-09-12/handler_post_0912_2.json, 1726117863832561>,
  'date': datetime.datetime(2024, 9, 12, 0, 0),
  'batch_number': 2},
 {'blob': <Blob: 0_raw_data, 2_xingtu/2024-09-12/handler_post_0912_3.json, 1726117867242406>,
  'date': datetime.datetime(2024, 9, 12, 0, 0),
  'batch_number': 3},
 {'blob': <Blo

In [4]:
processed_blobs = [{
    "blob": blob,
    "date": datetime.strptime(blob.name.split('/')[2], "%Y-%m-%d"),
    "batch_number": int(blob.name.split('/')[-1].replace(".parquet", "").split("_")[-1]),
} for blob in storage_client.list_blobs("3_staging_area",prefix="1_xingtu/") if table_name == blob.name.split("/")[1] and api_name in blob.name and len(blob.name.split("/")) == 4]
processed_blobs

[{'blob': <Blob: 3_staging_area, 1_xingtu/douyin_price/2024-09-13/handler_post_241009_0.parquet, 1728469231343006>,
  'date': datetime.datetime(2024, 9, 13, 0, 0),
  'batch_number': 0},
 {'blob': <Blob: 3_staging_area, 1_xingtu/douyin_price/2024-09-13/handler_post_241009_1.parquet, 1728469233446157>,
  'date': datetime.datetime(2024, 9, 13, 0, 0),
  'batch_number': 1},
 {'blob': <Blob: 3_staging_area, 1_xingtu/douyin_price/2024-10-02/handler_post_241009_0.parquet, 1728469234776362>,
  'date': datetime.datetime(2024, 10, 2, 0, 0),
  'batch_number': 0},
 {'blob': <Blob: 3_staging_area, 1_xingtu/douyin_price/2024-10-02/handler_post_241009_1.parquet, 1728469236516667>,
  'date': datetime.datetime(2024, 10, 2, 0, 0),
  'batch_number': 1},
 {'blob': <Blob: 3_staging_area, 1_xingtu/douyin_price/2024-10-02/handler_post_241009_2.parquet, 1728469238772536>,
  'date': datetime.datetime(2024, 10, 2, 0, 0),
  'batch_number': 2},
 {'blob': <Blob: 3_staging_area, 1_xingtu/douyin_price/2024-10-02/hand

In [5]:
to_process = []
for processing_blob in processing_blobs:
    if processing_blob["date"] >= datetime(2024, 9, 13):
        if processing_blob["batch_number"] not in [processed_blob["batch_number"] for processed_blob in processed_blobs if processing_blob["date"] == processed_blob["date"]]:
            to_process.append(processing_blob)
pprint(to_process)
print(len(to_process))

[{'batch_number': 0,
  'blob': <Blob: 0_raw_data, 2_xingtu/2024-10-11/handler_post_1011_0.json, 1728633838334445>,
  'date': datetime.datetime(2024, 10, 11, 0, 0)},
 {'batch_number': 1,
  'blob': <Blob: 0_raw_data, 2_xingtu/2024-10-11/handler_post_1011_1.json, 1728633842027146>,
  'date': datetime.datetime(2024, 10, 11, 0, 0)},
 {'batch_number': 2,
  'blob': <Blob: 0_raw_data, 2_xingtu/2024-10-11/handler_post_1011_2.json, 1728633842581655>,
  'date': datetime.datetime(2024, 10, 11, 0, 0)}]
3


In [6]:
def get_price_handler_post_data(raw_data):
    result = []
    for data in raw_data:
        profile_id = data["profile_id"]
        handler_post = data.get("handler_post")
        if handler_post:
            base_info = handler_post.get("base_info")
            if base_info:
                price_info = base_info.get("price_info")
                for price in price_info:
                    record = {}
                    record["video_type"] = price["video_type"]
                    record["settlement_type"] = price["settlement_type"]
                    record["is_open"] = price["is_open"]
                    record["price"] = price["price"]
                    record["influencer_id"] = profile_id
                    result.append(record)
    return result

In [7]:
for item in to_process:
    raw_data = json.loads(item["blob"].download_as_string())
    price_handler_post_data = get_price_handler_post_data(raw_data)
    df = pd.DataFrame(price_handler_post_data)
    df.to_parquet("gs://3_staging_area/1_xingtu/" + table_name + "/" + str(item["date"].date()) + "/" + api_name + "_" + today.strftime("%y%m%d") + "_" + str(item["batch_number"]) + ".parquet")